In [1]:
import pickle
#import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
import sklearn.metrics as metrics
from sklearn.cross_validation import StratifiedKFold

In [2]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Read in datafile

In [3]:
# Read data from files
df = pd.read_csv( 'labeledTrainData.tsv', header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0,  delimiter="\t", quoting=3 )

In [4]:
df[0:2]

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


In [6]:
print len(df)
print df["review"].size

25000
25000


In [7]:
df.columns = ['id', 'boolean_label', 'abstract']

In [8]:
df[0:2]

,id,boolean_label,abstract
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


### Read in indices objects from pickle

In [9]:
trainIndices = pickle.load(open('trainIndicesMR.p','rb'))
testIndices = pickle.load(open('testIndicesMR.p','rb'))

In [10]:
print(len(trainIndices[0]))
print(len(testIndices))

5
25


### Apply Word2Vec Inversion

In [11]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [12]:
def buildModels(train1, test1, docs, basemodel):
    
    score = []
    
    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
    inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
         
    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

    score.append(metrics.roc_auc_score(test1["boolean_label"], predictions))
    
    score.append(metrics.accuracy_score(test1["boolean_label"], predictions))
    
    score.append(metrics.f1_score(test1["boolean_label"], predictions))
    
    score.append(metrics.precision_score(test1["boolean_label"], predictions))
    
    score.append(metrics.recall_score(test1["boolean_label"], predictions))
    
    print(len(score))
    
    return score

In [14]:
a = datetime.datetime.now().replace(microsecond=0)

#CROSS VALIDATION
#r=0-24, our meta-folds

ROCs = []
accuracies = []
F1_scores = []
precisions = []
recalls = []

for r in range(len(trainIndices)):
    
    print("METAROUND", r+1)

    m = 1

    avgROC = []
    avgACC = []
    avgF1 = []
    avgPREC = []
    avgREC = []
    #i=0-4, our 5-folds
    for i in range(len(trainIndices[r])):

        print("ROUND", m)
        m = m + 1
        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[trainIndices[r][i]], df.iloc[testIndices[r][i]]


        sentences = []  # Initialize an empty list of sentences
        for j in range(len(train1)):
            review = train1['abstract'].iloc[j]
            sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        print "Parsing sentences from unlabeled set"
        for review in unlabeled_train["review"]:
            sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        print "Building and training w2v models"
        ## create a w2v learner
        basemodel = Word2Vec(
            sentences=None,
            size=100, #default
            window=5, #default
            workers=multiprocessing.cpu_count(),   # use your cores
            iter=3, # iter = sweeps of SGD through the data; more is better
            hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
        basemodel.build_vocab(sentences) 

        print "Parsing test sentences"
        # read in the test set as a list of a list of words

        docs = []
        for k in range(len(test1)):
            review = test1["abstract"].iloc[k]
            sents = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
            docs.append(sents)

        print "scoring test set"
        tup = buildModels(train1, test1, docs, basemodel)
        avgROC.append(tup[0])
        avgACC.append(tup[1])
        avgF1.append(tup[2])
        avgPREC.append(tup[3])
        avgREC.append(tup[4])


    print("average ROC:", sum(avgROC)/float(len(avgROC)))
    print("average ACC:", sum(avgACC)/float(len(avgACC)))
    print("average F1:", sum(avgF1)/float(len(avgF1)))
    print("average PREC:", sum(avgPREC)/float(len(avgPREC)))
    print("average REC:", sum(avgREC)/float(len(avgREC)))

    
    ROCs.append(sum(avgROC)/float(len(avgROC)))
    accuracies.append(sum(avgACC)/float(len(avgACC)))
    F1_scores.append(sum(avgF1)/float(len(avgF1)))
    precisions.append(sum(avgPREC)/float(len(avgPREC)))
    recalls.append(sum(avgREC)/float(len(avgREC)))
    
    print("--------------------------------------------------")

print("------------------------------------------------------")
print("FINAL RESULTS:")
print("ROCs:")
print(ROCs)
print("accuracies:")
print(accuracies)
print("F1-scores:")
print(F1_scores)
print("precisions:")
print(precisions)
print("recalls:")
print(recalls)
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('METAROUND', 1)
('ROUND', 1)


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /nlp-vm/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup 

Parsing sentences from unlabeled set


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to g

Building and training w2v models
Parsing test sentences
scoring test set
5
('ROUND', 2)
Parsing sentences from unlabeled set
Building and training w2v models
Parsing test sentences
scoring test set
5
('ROUND', 3)
Parsing sentences from unlabeled set
Building and training w2v models
Parsing test sentences
scoring test set
5
('ROUND', 4)
Parsing sentences from unlabeled set
Building and training w2v models
Parsing test sentences
scoring test set
5
('ROUND', 5)
Parsing sentences from unlabeled set
Building and training w2v models
Parsing test sentences
scoring test set
5
('average ROC:', 0.87836000000000014)
('average ACC:', 0.87836000000000003)
('average F1:', 0.88194954875633391)
('average PREC:', 0.8568498991876391)
('average REC:', 0.90871999999999997)
--------------------------------------------------
('METAROUND', 2)
('ROUND', 1)
Parsing sentences from unlabeled set
Building and training w2v models
Parsing test sentences
scoring test set
5
('ROUND', 2)
Parsing sentences from unlabel

In [16]:
import scipy.stats as st

print(" Accuracy")
print(st.t.interval(0.95, len(accuracies)-1, loc=np.mean(accuracies), scale=st.sem(accuracies)))
print("ROC")
print(st.t.interval(0.95, len(ROCs)-1, loc=np.mean(ROCs), scale=st.sem(ROCs)))
print("F1 score")
print(st.t.interval(0.95, len(F1_scores)-1, loc=np.mean(F1_scores), scale=st.sem(F1_scores)))
print("Precision")
print(st.t.interval(0.95, len(precisions)-1, loc=np.mean(precisions), scale=st.sem(precisions)))
print("Recall")
print(st.t.interval(0.95, len(recalls)-1, loc=np.mean(recalls), scale=st.sem(recalls)))


 Accuracy
(0.87634885238421156, 0.87753914761578855)
ROC
(0.87634885238421156, 0.87753914761578855)
F1 score
(0.88045671896488131, 0.88151361059297439)
Precision
(0.85162322390431955, 0.85486273833740234)
Recall
(0.90901198172081599, 0.91267761827918403)


In [13]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences